# Equations of motion for the cat-bot

Do imports.

In [2]:
import sympy as sym
import numpy as np
from IPython.display import display, Markdown
from sympy.physics import mechanics
mechanics.init_vprinting()

Define variables and constants as symbols.

In [3]:
# Time
t = sym.Symbol('t')

# Horizontal position of wheel axle
zeta = mechanics.dynamicsymbols('zeta', real=True)

# Angle of body from vertical (positive means leaning forward)
theta = mechanics.dynamicsymbols('theta', real=True)

# Torque applied by the body on the wheel
tau = sym.symbols('tau', real=True)

# Wheel parameters
#   m_w  mass
#   J_w  moment of inertia
#   r_w  radius
m_w, J_w, r_w = sym.symbols('m_w, J_w, r_w', real=True, positive=True)

# Body parameters
#   m_b  mass
#   J_b  moment of inertia (about center-of-mass)
#   r_b  distance between wheel axle and body center-of-mass
m_b, J_b, r_b = sym.symbols('m_b, J_b, r_b', real=True, positive=True)

# Acceleration of gravity
g = sym.symbols('g', real=True, positive=True)

Compute Lagrangian.

In [4]:
# Position of wheel and body
p_w = sym.Matrix([zeta, r_w])
p_b = sym.Matrix([zeta + r_b * sym.sin(theta), r_w + r_b * sym.cos(theta)])

# Linear velocity of wheel and body
v_w = p_w.diff(t)
v_b = p_b.diff(t)

# Angular velocity of wheel (assume rolling without slipping on flat ground) and body
omega_w = zeta.diff(t) / r_w
omega_b = theta.diff(t)

# Kinetic and potential energy
T = (m_w * v_w.dot(v_w) + m_b * v_b.dot(v_b) + J_w * omega_w**2 + J_b * omega_b**2) / 2
V = (m_w * p_w[1] * g) + (m_b * p_b[1] * g)

# Lagrangian
L = sym.Matrix([sym.simplify(T - V)])

Compute and show the matrices $M(q)$, $N(q, \dot{q})$, and $F(q)$ for which the equations of motion can be expressed as

$$ M(q) \ddot{q} + N(q, \dot{q}) = F(q) r $$

where

$$ q = \begin{bmatrix} \zeta \\ \theta \end{bmatrix} \qquad\text{and}\qquad r = \begin{bmatrix} \tau \end{bmatrix}. $$

In [5]:
# Generalized coordinates
q = sym.Matrix([zeta, theta])

# Generalized velocities
v = q.diff(t)

# Coefficients in the equations of motion
M = sym.simplify(L.jacobian(v).jacobian(v))
N = sym.simplify(L.jacobian(v).jacobian(q) @ v - L.jacobian(q).T)
F = sym.simplify(sym.Matrix([(zeta / r_w) - theta]).jacobian(q).T)

# Show results
display(Markdown(f'$$ M(q) = {mechanics.mlatex(M)} $$'))
display(Markdown(f'$$ N(q, \\dot{{q}}) = {mechanics.mlatex(N)} $$'))
display(Markdown(f'$$ F(q) = {mechanics.mlatex(F)} $$'))

$$ M(q) = \left[\begin{matrix}\frac{J_{w}}{r_{w}^{2}} + m_{b} + m_{w} & m_{b} r_{b} \cos{\left(\theta \right)}\\m_{b} r_{b} \cos{\left(\theta \right)} & J_{b} + m_{b} r_{b}^{2}\end{matrix}\right] $$

$$ N(q, \dot{q}) = \left[\begin{matrix}- m_{b} r_{b} \sin{\left(\theta \right)} \dot{\theta}^{2}\\- g m_{b} r_{b} \sin{\left(\theta \right)}\end{matrix}\right] $$

$$ F(q) = \left[\begin{matrix}\frac{1}{r_{w}}\\-1\end{matrix}\right] $$

In [6]:
# m matrix
m = sym.Matrix([q,v])
m_dot = m.diff(t)

# Torque Matrix
n = sym.Matrix([tau])

# m_dot = f(m,n) vector
f = sym.Matrix([ v , M.inv() @ F @ n - M.inv() @ N ])

# finding the A and B matrices for linearization: x_dot = A@x + B@u
A_sym = sym.simplify(f.jacobian([m]))
B_sym = sym.simplify(f.jacobian([n]))
A_num = sym.lambdify((zeta,theta,zeta.diff(t),theta.diff(t),tau,r_w, m_w, J_w, r_b, m_b, J_b, g), A_sym)
B_num = sym.lambdify((zeta,theta,zeta.diff(t),theta.diff(t),tau,r_w, m_w, J_w, r_b, m_b, J_b, g), B_sym)
A = A_num(0.,0.,0.,0.,0.,.325,2.4,.12675,0.3,12.,.8,9.81)
B = B_num(0.,0.,0.,0.,0.,.325,2.4,.12675,0.3,12.,.8,9.81)



In [7]:
def eig_check(K,n):
    F = A - B @ K
    E = F.eigenvals().keys()
    print('Eigenvalues from K',n)
    for i in E:
        print(i)


In [8]:
K1,n1 = sym.Matrix([[ -10 , -1000 , -10 , -50 ]]) , 1
K2,n2 = sym.Matrix([[ -20 , -1000 , -10 , -50 ]]) , 2
K3,n3 = sym.Matrix([[ -30 , -1000 , -10 , -50 ]]) , 3
K4,n4 = sym.Matrix([[ -40 , -1000 , -10 , -50 ]]) , 4
K5,n5 = sym.Matrix([[ -50 , -1000 , -10 , -50 ]]) , 5
K6,n6 = sym.Matrix([[ -60 , -1000 , -10 , -50 ]]) , 6
K7,n7 = sym.Matrix([[ -70 , -1000 , -10 , -50 ]]) , 7
K8,n8 = sym.Matrix([[ -80 , -1000 , -10 , -50 ]]) , 8
K9,n9 = sym.Matrix([[ -90 , -1000 , -10 , -50 ]]) , 9
K10,n10 = sym.Matrix([[ -100 , -1000 , -10 , -50 ]]) , 10

K_list = [K1,K2,K3,K4,K5,K6,K7,K8,K9,K10]
n_list = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10]

print('pitch = -1000 , speed = -10 , pitch rate = -50')
print('Varying position multiplier from -10 to -100')

for i in range(0,len(K_list)):
    eig_check(K_list[i],n_list[i])

pitch = -1000 , speed = -10 , pitch rate = -50
Varying position multiplier from -10 to -100
Eigenvalues from K 1
-0.0199139594542572 - 0.203534535406462*I
-0.0199139594542572 + 0.203534535406462*I
-37.8588739193336 - 12.4127956318801*I
-37.8588739193336 + 12.4127956318801*I
Eigenvalues from K 2
-0.0189766778038428 - 0.289108210474122*I
-0.0189766778038428 + 0.289108210474122*I
-37.859811200984 - 12.1808356705608*I
-37.859811200984 + 12.1808356705608*I
Eigenvalues from K 3
-0.0180253935246131 - 0.355021872500645*I
-0.0180253935246131 + 0.355021872500645*I
-37.8607624852633 - 11.9444038551307*I
-37.8607624852633 + 11.9444038551307*I
Eigenvalues from K 4
-0.0170598828285482 - 0.41085306290662*I
-0.0170598828285482 + 0.41085306290662*I
-37.8617279959593 - 11.7032297655975*I
-37.8617279959593 + 11.7032297655975*I
Eigenvalues from K 5
-0.0160799178901324 - 0.460288752661166*I
-0.0160799178901324 + 0.460288752661166*I
-37.8627079608977 - 11.4570145530233*I
-37.8627079608977 + 11.4570145530233

In [9]:
K1,n1 = sym.Matrix([[ -50 , -100 , -10 , -50 ]]) , 1
K2,n2 = sym.Matrix([[ -50 , -200 , -10 , -50 ]]) , 2
K3,n3 = sym.Matrix([[ -50 , -300 , -10 , -50 ]]) , 3
K4,n4 = sym.Matrix([[ -50 , -400 , -10 , -50 ]]) , 4
K5,n5 = sym.Matrix([[ -50 , -500 , -10 , -50 ]]) , 5
K6,n6 = sym.Matrix([[ -50 , -600 , -10 , -50 ]]) , 6
K7,n7 = sym.Matrix([[ -50 , -700 , -10 , -50 ]]) , 7
K8,n8 = sym.Matrix([[ -50 , -800 , -10 , -50 ]]) , 8
K9,n9 = sym.Matrix([[ -50 , -900 , -10 , -50 ]]) , 9
K10,n10 = sym.Matrix([[ -50 , -1000 , -10 , -50 ]]) , 10

K_list = [K1,K2,K3,K4,K5,K6,K7,K8,K9,K10]
n_list = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10]

print('position = -50 , speed = -10 , pitch rate = -50')
print('Varying pitch from -100 to -1000')

for i in range(0,len(K_list)):
    eig_check(K_list[i],n_list[i])

position = -50 , speed = -10 , pitch rate = -50
Varying pitch from -100 to -1000
Eigenvalues from K 1
0.339990083030691 - 1.44606462596429*I
0.339990083030691 + 1.44606462596429*I
-2.02140240441116
-74.4161535192260
Eigenvalues from K 2
0.0497929614886532 - 1.10774856809193*I
0.0497929614886532 + 1.10774856809193*I
-3.74357417982127
-72.1135875007318
Eigenvalues from K 3
-0.00844458503059682 - 0.884183585561564*I
-0.00844458503059682 + 0.884183585561564*I
-6.09607813489974
-69.6446084526148
Eigenvalues from K 4
-0.0200494380141577 - 0.752364802818626*I
-0.0200494380141577 + 0.752364802818626*I
-8.75058642713875
-66.9668904544087
Eigenvalues from K 5
-0.0219181698009541 - 0.665436298520583*I
-0.0219181698009541 + 0.665436298520583*I
-11.6973494353649
-64.0163899826089
Eigenvalues from K 6
-0.0212887007376947 - 0.602932007483563*I
-0.0212887007376947 + 0.602932007483563*I
-15.0274441908711
-60.6875541652292
Eigenvalues from K 7
-0.0200019819932709 - 0.555268232712778*I
-0.020001981993270

In [10]:
K1,n1 = sym.Matrix([[ -50 , -1000 , -1 , -50 ]]) , 1
K2,n2 = sym.Matrix([[ -50 , -1000 , -2.5 , -50 ]]) , 2
K3,n3 = sym.Matrix([[ -50 , -1000 , -5 , -50 ]]) , 3
K4,n4 = sym.Matrix([[ -50 , -1000 , -7.5 , -50 ]]) , 4
K5,n5 = sym.Matrix([[ -50 , -1000 , -10 , -50 ]]) , 5
K6,n6 = sym.Matrix([[ -50 , -1000 , -12.5 , -50 ]]) , 6
K7,n7 = sym.Matrix([[ -50 , -1000 , -15 , -50 ]]) , 7
K8,n8 = sym.Matrix([[ -50 , -1000 , -17.5 , -50 ]]) , 8
K9,n9 = sym.Matrix([[ -50 , -1000 , -20 , -50 ]]) , 9
K10,n10 = sym.Matrix([[ -50 , -1000 , -25 , -50 ]]) , 10

K_list = [K1,K2,K3,K4,K5,K6,K7,K8,K9,K10]
n_list = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10]

print('position = -50 , pitch = -1000 , pitch rate = -50')
print('Varying speed from -1 to -25')

for i in range(0,len(K_list)):
    eig_check(K_list[i],n_list[i])

position = -50 , pitch = -1000 , pitch rate = -50
Varying speed from -1 to -25
Eigenvalues from K 1
0.00334686343307226 - 0.460119831942329*I
0.00334686343307226 + 0.460119831942329*I
-32.1336099893940
-48.7908219132748
Eigenvalues from K 2
0.000114378849504666 - 0.460208820292585*I
0.000114378849504666 + 0.460208820292585*I
-34.1114844634204
-45.9464554003768
Eigenvalues from K 3
-0.00527798179761888 - 0.460303166275633*I
-0.00527798179761888 + 0.460303166275633*I
-39.3068883464977 - 4.62839662621611*I
-39.3068883464977 + 4.62839662621611*I
Eigenvalues from K 4
-0.0106761839910061 - 0.460329880761579*I
-0.0106761839910061 + 0.460329880761579*I
-38.5848009195506 - 8.76633586645897*I
-38.5848009195506 + 8.76633586645897*I
Eigenvalues from K 5
-0.0160799178901324 - 0.460288752661166*I
-0.0160799178901324 + 0.460288752661166*I
-37.8627079608977 - 11.4570145530233*I
-37.8627079608977 + 11.4570145530233*I
Eigenvalues from K 6
-0.0214888709979264 - 0.460179556228745*I
-0.0214888709979264 + 0

In [11]:
K1,n1 = sym.Matrix([[ -50 , -1000 , -10 , -10 ]]) , 1
K2,n2 = sym.Matrix([[ -50 , -1000 , -10 , -20 ]]) , 2
K3,n3 = sym.Matrix([[ -50 , -1000 , -10 , -30 ]]) , 3
K4,n4 = sym.Matrix([[ -50 , -1000 , -10 , -40 ]]) , 4
K5,n5 = sym.Matrix([[ -50 , -1000 , -10 , -50 ]]) , 5
K6,n6 = sym.Matrix([[ -50 , -1000 , -10 , -60 ]]) , 6
K7,n7 = sym.Matrix([[ -50 , -1000 , -10 , -70 ]]) , 7
K8,n8 = sym.Matrix([[ -50 , -1000 , -10 , -80 ]]) , 8
K9,n9 = sym.Matrix([[ -50 , -1000 , -10 , -90 ]]) , 9
K10,n10 = sym.Matrix([[ -50 , -1000 , -10 , -100 ]]) , 10

K_list = [K1,K2,K3,K4,K5,K6,K7,K8,K9,K10]
n_list = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10]

print('position = -50 , pitch = -1000 , speed = -10')
print('Varying pitch rate from -10 to -100')

for i in range(0,len(K_list)):
    eig_check(K_list[i],n_list[i])

position = -50 , pitch = -1000 , speed = -10
Varying pitch rate from -10 to -100
Eigenvalues from K 1
-0.0204738358713015 + 0.459819231012692*I
-0.0204738358713015 - 0.459819231012692*I
-5.2618782206743 - 39.2321951871649*I
-5.2618782206743 + 39.2321951871649*I
Eigenvalues from K 2
-13.4120823343559 - 37.233852290104*I
-0.0193786777502626 - 0.459956211388621*I
-0.0193786777502626 + 0.459956211388621*I
-13.4120823343559 + 37.233852290104*I
Eigenvalues from K 3
-0.0182811066450867 - 0.460080169429803*I
-0.0182811066450867 + 0.460080169429803*I
-21.5622888610216 - 33.1779179416734*I
-21.5622888610216 + 33.1779179416734*I
Eigenvalues from K 4
-0.0171814199774254 - 0.460191036115753*I
-0.0171814199774254 + 0.460191036115753*I
-29.7124975032499 - 26.1229532667378*I
-29.7124975032499 + 26.1229532667378*I
Eigenvalues from K 5
-0.0160799178901324 - 0.460288752661166*I
-0.0160799178901324 + 0.460288752661166*I
-37.8627079608977 - 11.4570145530233*I
-37.8627079608977 + 11.4570145530233*I
Eigenval

In [25]:
K1,n1 = sym.Matrix([[ -200 , -1000 , -200 , -200 ]]) , 1
K2,n2 = sym.Matrix([[ -50 , -1000 , -10 , -50 ]]) , 2
K3,n3 = sym.Matrix([[ -60 , -1000 , -10 , -50 ]]) , 3
K4,n4 = sym.Matrix([[ -70 , -1000 , -10 , -50 ]]) , 4
K5,n5 = sym.Matrix([[ -80 , -1000 , -10 , -50 ]]) , 5
K6,n6 = sym.Matrix([[ -40 , -1000 , -30 , -50 ]]) , 6
K7,n7 = sym.Matrix([[ -50 , -1000 , -30 , -50 ]]) , 7
K8,n8 = sym.Matrix([[ -60 , -1000 , -30 , -50 ]]) , 8
K9,n9 = sym.Matrix([[ -70 , -1000 , -30 , -50 ]]) , 9
K10,n10 = sym.Matrix([[ 400 , -1000 , 100 , -200 ]]) , 10

K_list = [K1,K2,K3,K4,K5,K6,K7,K8,K9,K10]
n_list = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10]

#print('position = -50 , pitch = -1000 , speed = -10')
#print('Varying pitch rate from -10 to -100')

for i in range(0,len(K_list)):
    eig_check(K_list[i],n_list[i])

Eigenvalues from K 1
-0.427914850578779 - 0.915400086060159*I
-0.427914850578779 + 0.915400086060159*I
-6.37231378966795
-204.065938770999
Eigenvalues from K 2
-0.0160799178901324 - 0.460288752661166*I
-0.0160799178901324 + 0.460288752661166*I
-37.8627079608977 - 11.4570145530233*I
-37.8627079608977 + 11.4570145530233*I
Eigenvalues from K 3
-0.015085266766786 - 0.505212931718193*I
-0.015085266766786 + 0.505212931718193*I
-37.8637026120211 - 11.2054265719992*I
-37.8637026120211 + 11.2054265719992*I
Eigenvalues from K 4
-0.0140756933176869 - 0.54674168912121*I
-0.0140756933176869 + 0.54674168912121*I
-37.8647121854702 - 10.9480961107847*I
-37.8647121854702 + 10.9480961107847*I
Eigenvalues from K 5
-0.0130509571209523 - 0.585600547730031*I
-0.0130509571209523 + 0.585600547730031*I
-37.8657369216669 - 10.6846089802484*I
-37.8657369216669 + 10.6846089802484*I
Eigenvalues from K 6
-0.0601399795762503 - 0.407466790904299*I
-0.0601399795762503 + 0.407466790904299*I
-32.0851341011817 - 23.14918

Define and show the numerical value of constants (consistant with the URDF file).

In [13]:
##########################################
# Temporary variables that can be ignored

# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

##########################################
# Parameters

# Define them
params = {
    r_w: r,
    m_w: 2 * mw,
    J_w: 2 * Jw,
    r_b: h,
    m_b: mb,
    J_b: Jby,
    g: 9.81,
}

# Show them
s = ''
for key, val in params.items():
    s += fr'{key} &= {mechanics.mlatex(val)} \\ '
s = s[:-3]
display(Markdown(fr'$$ \begin{{align*}}{s}\end{{align*}} $$'))

$$ \begin{align*}r_w &= 0.325 \\ m_w &= 2.4 \\ J_w &= 0.12675 \\ r_b &= 0.3 \\ m_b &= 12.0 \\ J_b &= 0.8 \\ g &= 9.81 \end{align*} $$

In [14]:
# f_num = sym.lambdify([q, v, tau], f)
# f_num(q_e, v_e, tau_e)

# M_num = sym.lambdify([theta,r_w, m_w, J_w, r_b, m_b, J_b, g], M)
# N_num = sym.lambdify([zeta, theta, zeta.diff(t), theta.diff(t), r_w, m_w, J_w, r_b, m_b, J_b, g], N)
# F_num = sym.lambdify([r_w], F)
# zeta_e, theta_e = sym.symbols('zeta_e, theta_e')
# M_num(1,1,1,1,1,1,11,1,1,1,1) , N_num(1,1,1,1,1,1,1,1,1,1,1) , F_num(1,1,1,1,1,1,1,1,1,1,1)

In [15]:
rw = r
mw = 2 * mw
Jw = 2 * Jw
rb = h
mb = mb
Jb = Jby
g  = 9.81

M = sym.Matrix([ [ Jw / (rw**2) + mb + mw , mb*rb*sym.cos(theta) ] ,
                 [ mb*rb*sym.cos(theta)   , Jb + mb*(rb**2)      ] ])
N = sym.Matrix([ [ -mb*rb*sym.sin(theta)*(theta.diff(t)**2) ] ,
                 [ -g*mb*rb*sym.sin(theta)                  ] ])
F = sym.Matrix([ 1/rw , -1 ])

In [16]:
# m matrix
m = sym.Matrix([q,v])
m_dot = m.diff(t)

# Torque Matrix
n = sym.Matrix([tau])

# m_dot = f(m,n) vector
f = sym.simplify(sym.Matrix([ v , M.inv() @ F @ n - M.inv() @ N ]))

# finding the A and B matrices for linearization: x_dot = A@x + B@u
A_sym = sym.simplify(f.jacobian([m]))
B_sym = sym.simplify(f.jacobian([n]))

A_sym , B_sym

⎛⎡0                                                                            ↪
⎜⎢                                                                             ↪
⎜⎢0                                                                            ↪
⎜⎢                                                                             ↪
⎜⎢                          ⎛                     ⎛         2            ⎞ ⎛   ↪
⎜⎢         1.64766347715557⋅⎝- 0.0301349241001712⋅⎝12.96⋅cos (θ) - 29.328⎠⋅⎝τ⋅ ↪
⎜⎢0        ─────────────────────────────────────────────────────────────────── ↪
⎜⎢                                                                             ↪
⎜⎢                                                                             ↪
⎜⎢                                                                             ↪
⎜⎢                    ⎛                   ⎛         2            ⎞ ⎛           ↪
⎜⎢   1.64766347715557⋅⎝0.0301349241001712⋅⎝12.96⋅cos (θ) - 29.328⎠⋅⎝τ⋅(11.0769 ↪
⎜⎢0  ───────────────────────